In [21]:
!gpustat

> NVIDIA A100-SXM4-40GB | 51'C, 100 % | 24406 / 40960 MB |995050220[2033441551366007819906:006:0011122043455667788191001114012203336[20[48445[49[1756428[16770883[2799158

In [22]:
1500/60

25.0

In [28]:
import os
import logging
import torch
import subprocess

# (1) Make sure your logger is configured to write to file:
logging.basicConfig(
    filename="training.log",
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
)

def log_gpu_info():
    # (2) How many CUDA-visible devices?
    ngpu = torch.cuda.device_count()
    logging.info(f"CUDA_VISIBLE_DEVICES reports {ngpu} device(s).")
    print("hi")
    for i in range(ngpu):
        props = torch.cuda.get_device_properties(i)
        # Major/minor, total memory, multiProcessor count, name
        logging.info(
            f"  – GPU {i}: {props.name}  "
            f"(SM {props.major}.{props.minor}, "
            f"{props.total_memory/1024**3:.1f} GB VRAM, "
            f"{props.multi_processor_count} SMs)"
        )
        print(            f"  – GPU {i}: {props.name}  "
            f"(SM {props.major}.{props.minor}, "
            f"{props.total_memory/1024**3:.1f} GB VRAM, "
            f"{props.multi_processor_count} SMs)")

    # (3) Log CUDA and cuDNN versions from PyTorch
    logging.info(f"PyTorch version: {torch.__version__}")
    logging.info(f"CUDA build version: {torch.version.cuda}")
    logging.info(f"cuDNN version: {torch.backends.cudnn.version()}")
    
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA build version: {torch.version.cuda}")
    print(f"cuDNN version: {torch.backends.cudnn.version()}")

    # (4) If you want the actual driver, MIG, topology info via nvidia-smi:
    try:
        smi = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=index,name,driver_version,memory.total,mig.mode.current", "--format=csv,noheader"]
        ).decode("utf-8")
        logging.info("`nvidia-smi --query-gpu=` output:\n" + smi.strip())
        print("`nvidia-smi --query-gpu=` output:\n" + smi.strip())
    except Exception as e:
        logging.warning(f"Could not run nvidia-smi: {e}")

if __name__ == "__main__":
    # (5) Call this before you create any CUDA tensors or torch.distributed.init_process_group
    log_gpu_info()



hi
  – GPU 0: NVIDIA A100-SXM4-40GB  (SM 8.0, 39.4 GB VRAM, 108 SMs)
  – GPU 1: NVIDIA A100-SXM4-40GB  (SM 8.0, 39.4 GB VRAM, 108 SMs)
  – GPU 2: NVIDIA A100-SXM4-40GB  (SM 8.0, 39.4 GB VRAM, 108 SMs)
  – GPU 3: NVIDIA A100-SXM4-40GB  (SM 8.0, 39.4 GB VRAM, 108 SMs)
PyTorch version: 2.0.0+cu117
CUDA build version: 11.7
cuDNN version: 8500
`nvidia-smi --query-gpu=` output:
0, NVIDIA A100-SXM4-40GB, 550.54.15, 40960 MiB, Disabled
1, NVIDIA A100-SXM4-40GB, 550.54.15, 40960 MiB, Disabled
2, NVIDIA A100-SXM4-40GB, 550.54.15, 40960 MiB, Disabled
3, NVIDIA A100-SXM4-40GB, 550.54.15, 40960 MiB, Disabled


In [1]:
import subprocess, textwrap, datetime, pathlib, json, shlex

def sh(cmd, timeout=30):
    "Run *cmd* (string or list) and return decoded stdout."
    if isinstance(cmd, str):
        cmd = shlex.split(cmd)
    out = subprocess.check_output(cmd, timeout=timeout)
    return out.decode()

def log_block(title, cmd, timeout=30):
    print(f"\n\n### {title}\n```bash\n{cmd}\n```\n```\n{sh(cmd, timeout)}\n```")


In [2]:
log_block("uname -a", "uname -a")
log_block("uptime", "uptime -p")
log_block("NUMA layout", "numactl --hardware")




### uname -a
```bash
uname -a
```
```
Linux jupyter-msalehjahromi 5.4.0-177-generic #197-Ubuntu SMP Thu Mar 28 22:45:47 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux

```


### uptime
```bash
uptime -p
```
```
up 1 year, 6 weeks, 1 day, 19 hours, 47 minutes

```


FileNotFoundError: [Errno 2] No such file or directory: 'numactl'

In [10]:
!uname -a


Linux jupyter-msalehjahromi 5.4.0-177-generic #197-Ubuntu SMP Thu Mar 28 22:45:47 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux


In [12]:
!lscpu | egrep 'NUMA|Socket|Node'


Socket(s):                          2
NUMA node(s):                       8
NUMA node0 CPU(s):                  0-15,128-143
NUMA node1 CPU(s):                  16-31,144-159
NUMA node2 CPU(s):                  32-47,160-175
NUMA node3 CPU(s):                  48-63,176-191
NUMA node4 CPU(s):                  64-79,192-207
NUMA node5 CPU(s):                  80-95,208-223
NUMA node6 CPU(s):                  96-111,224-239
NUMA node7 CPU(s):                  112-127,240-255


In [13]:
!mkdir -p /rsrch1/ip/$USER/nfs_test
!dd if=/dev/urandom of=/rsrch1/ip/$USER/nfs_test/tmp_nfs.dat bs=1M count=1024 status=none
!dd if=/rsrch1/ip/$USER/nfs_test/tmp_nfs.dat of=/dev/null bs=1M count=1024 status=none


In [8]:
# 1 GiB temp file on NFS
log_block("Create 1-GiB NFS test file", 
          "dd if=/dev/urandom of=/rsrch1/ip/$USER/tmp_nfs.dat bs=1M count=1024 status=none")

# Read speed NFS → RAM
log_block("Read speed NFS → /dev/null", 
          "dd if=/rsrch1/ip/$USER/tmp_nfs.dat of=/dev/null bs=1M count=1024 status=progress")

# 1 GiB temp file on local NVMe (choose the fast SSD you have permission for)
log_block("Create 1-GiB NVMe test file", 
          "dd if=/dev/urandom of=/local_nvme/tmp_nvme.dat bs=1M count=1024 status=none")

# Read speed NVMe → RAM
log_block("Read speed NVMe → /dev/null", 
          "dd if=/local_nvme/tmp_nvme.dat of=/dev/null bs=1M count=1024 status=progress")


dd: failed to open '/rsrch1/ip/$USER/tmp_nfs.dat': No such file or directory


CalledProcessError: Command '['dd', 'if=/dev/urandom', 'of=/rsrch1/ip/$USER/tmp_nfs.dat', 'bs=1M', 'count=1024', 'status=none']' returned non-zero exit status 1.

In [7]:
log_block("NIC statistics (1 s)", "sar -n DEV 1 1 | grep -E 'mlx5_[01]'")
log_block("GPU↔NIC topology slice", 
          "nvidia-smi topo -m | grep -E 'GPU[56]|NIC[01]'")


FileNotFoundError: [Errno 2] No such file or directory: 'sar'

In [9]:
import torch, time, math
size = (1024,1024,128)      # ≈512 MiB float32
def measure_hd(gpu=0):
    cpu = torch.empty(size, dtype=torch.float32, pin_memory=True)
    torch.cuda.synchronize(gpu)
    t0=time.time(); dev=cpu.to(f"cuda:{gpu}", non_blocking=True); torch.cuda.synchronize(gpu)
    h2d=cpu.numel()*4/(time.time()-t0)/1024**3
    t0=time.time(); _=dev.to("cpu", non_blocking=True); torch.cuda.synchronize(gpu)
    d2h=cpu.numel()*4/(time.time()-t0)/1024**3
    print(f"GPU{gpu}  H→D {h2d:5.1f} GB/s   D→H {d2h:5.1f} GB/s")
    
measure_hd(5)
measure_hd(6)


RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
